In [102]:
%cd C:\Users\yukir\Documents\GitHub\Sign_Recognition\py
from basic_preprocessing import *
from deeplearning_check import *
from machine_learning import *

gpu_check()
set_global_determinism(seed=SEED)

C:\Users\yukir\Documents\GitHub\Sign_Recognition\py
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16916083113231338628
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15293416635148734156
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9883535296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11371599911884087136
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13239020461562938042
physical_device_desc: "device: XLA_GPU device"
]
tf 2.2.0
keras 2.3.0-tf
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라


In [103]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [104]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [105]:
model =hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [106]:
def show(img):
    #사이즈
    plt.figure(figsize = (10,8))
    #xticks/yticks - 눈금표
    plt.xticks([])
    plt.yticks([])
    #코랩에서 안돌아감 주의
    plt.imshow(img, cmap= 'gray')
    plt.show()

In [107]:
def vid_info(path, codec, name):
    cap = cv2.VideoCapture(path)
    print(cap)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

    #재생할 파일의 높이 얻기
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    #재생할 파일의 프레임 레이트 얻기
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    #codec
    fourcc = cv2.VideoWriter_fourcc(*codec)
    
    #filename
    filename = name+'.mp4'
    
    #out 
    out = cv2.VideoWriter(filename, fourcc, fps, (int(width), int(height)))
    
    print('cap {0}, width {1}, height {2}, fps {3}'.format(cap, width, height, fps))
    print('codec {0}', 'filename {1}'.format(fourcc, filename))
    
    return cap, width, height, fps, fourcc, filename, out


In [108]:
path = "C:\\Users\\yukir\\Documents\\Monicas_workspace\\pose_estimation\\IVE 아이브 'LOVE DIVE' DANCE PRACTICE.mp4"

In [120]:
cap, width, height, fps, fourcc, filename, out = vid_info(path, 'XVID', 'test6')

<VideoCapture 0000021FA1988590>
cap <VideoCapture 0000021FA1988590>, width 1280.0, height 720.0, fps 23.976023976023978
codec {0} filename test6.mp4


In [110]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [111]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [112]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [113]:
#function to loop through each person detected and render

def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [114]:
width, height

(1280.0, 720.0)

In [115]:
%cd C:\Users\yukir\Documents\Monicas_workspace\pose_estimation

C:\Users\yukir\Documents\Monicas_workspace\pose_estimation


In [116]:
cap = cv2.VideoCapture(path)
k = 0
img_list = []
while True:
    ret, frame = cap.read()
    k+=1
    if k<200:
        # show(frame)
        img_list.append(frame)
        # break
    else:
        break

In [117]:
img_list = img_list[50:]
# print(img_list)

In [118]:
img_list[1].shape

(720, 1280, 3)

In [121]:
cap = cv2.VideoCapture("C:\\Users\\yukir\\Documents\\Monicas_workspace\\pose_estimation\\test2.mp4")
k = 0

while True:
    ret, frame = cap.read()
    k+=1
    try: 
        if 50 <= k <=3000:
            
            # print(k)
            # Resize image
            img = frame.copy()
            # show(img)
            ## 사이즈 모델이 32의 배수여야하고 256pixel이상이여야함
            ### ratio 따라서 reshape해야함 
            img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 352,608)
            input_img = tf.cast(img, dtype=tf.int32)
        #
            # Detection section
            results = movenet(input_img)

            #rendering
            keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
            
            # Render keypoints 
            loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
            out.write(frame)

    except:
        break
        
out.release()
print('released')

released
